In [1]:
import pickle as pk
import requests

import bs4 as bs
from bs4 import XMLParsedAsHTMLWarning
import warnings

warnings.filterwarnings('ignore', category=XMLParsedAsHTMLWarning)

## Get Data

In [2]:
url = 'https://github.com/fadeeva/EDGAR_research/raw/refs/heads/master/data/reports.pickle'
response = requests.get(url)
reports = pk.loads(response.content)

len(reports.keys())

496

### Get Random Reports

In [4]:
import random

def get_rand_reports(tickers:list=[], number_of_reports:int=5)->dict:
    tickers = random.sample(list(reports.keys()), 5) if not tickers else tickers
    data = {ticker: random.sample(reports[ticker], number_of_reports) for ticker in tickers}

    return data

In [6]:
rand_reports = get_rand_reports()
rand_reports

{'QCOM': ['https://www.sec.gov/Archives/edgar/data/804328/000080432824000017/qcom-20231224.htm',
  'https://www.sec.gov/Archives/edgar/data/804328/000172894922000012/qcom-20211226.htm',
  'https://www.sec.gov/Archives/edgar/data/804328/000080432823000006/qcom-20221225.htm',
  'https://www.sec.gov/Archives/edgar/data/804328/000172894921000066/qcom-20210627.htm',
  'https://www.sec.gov/Archives/edgar/data/804328/000080432825000031/qcom-20250330.htm'],
 'KKR': ['https://www.sec.gov/Archives/edgar/data/1843212/000184321222000012/kahc-20220630.htm',
  'https://www.sec.gov/Archives/edgar/data/2012839/000201283924000009/ebdc-20240630.htm',
  'https://www.sec.gov/Archives/edgar/data/1631596/000163159620000032/kref-20200930.htm',
  'https://www.sec.gov/Archives/edgar/data/1404912/000140491221000028/kkr-20210630.htm',
  'https://www.sec.gov/Archives/edgar/data/1631596/000163159623000033/kref-20230630.htm'],
 'CINF': ['https://www.sec.gov/Archives/edgar/data/20286/000002028621000033/cinf-20210331

In [7]:
aapl_reports = get_rand_reports(['AAPL'], 10)
aapl_reports

{'AAPL': ['https://www.sec.gov/Archives/edgar/data/320193/000032019322000059/aapl-20220326.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019325000008/aapl-20241228.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019325000057/aapl-20250329.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019324000006/aapl-20231230.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019322000007/aapl-20211225.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019323000064/aapl-20230401.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019321000065/aapl-20210626.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019321000010/aapl-20201226.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019323000077/aapl-20230701.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019323000006/aapl-20221231.htm']}

### Get Corpus

In [ ]:
def get_report_corpus(url:str)->bs:
    headers={
        'User-Agent':'Dummy Company jane.dow@dummy.com',
        'Accept-Encoding':'gzip, deflate',
        'Host':'www.sec.gov'
    }
    response = requests.get(url, headers=headers, allow_redirects=True)

    return bs.BeautifulSoup(response.text, features='lxml')


In [ ]:
corpus = get_report_corpus('https://www.sec.gov/Archives/edgar/data/320193/000032019325000057/aapl-20250329.htm')

In [ ]:
ticker, date = corpus.find('title').text.split('-')
ticker, date

('aapl', '20250329')

In [ ]:
corpus.find(id='f-5').text # Commission File Numbe

'001-36743'

In [ ]:
corpus.find(id='f-8').text # I.R.S. Employer Identification No

'94-2404110'

## Consolidated Balance Sheet

## Management’s Discussion and Analysis of Financial Condition and Results of Operations

In [ ]:
title_vars = [
    'ITEM 2. Management’s Discussion and Analysis of Financial Condition and Results of Operations',
    'Management’s Discussion and Analysis of Financial Condition and Results of Operations',
]

In [ ]:
# i056866be11a54295b8c21e0877b67331_67

In [ ]:
def get_ids(corpus):
    mdaf = corpus.find_all('span')
    tag = [tag for tag in mdaf if tag.string in title_vars][0]

    return tag.find('a').get('href')[1:], tag.parent.parent.parent.next_sibling.find('a').get('href')[1:]


In [ ]:
get_ids(corpus)

('i056866be11a54295b8c21e0877b67331_67',
 'i056866be11a54295b8c21e0877b67331_145')

In [ ]:
reports['LHX'][:3]

['https://www.sec.gov/Archives/edgar/data/202058/000020205825000101/hrs-20250328.htm',
 'https://www.sec.gov/Archives/edgar/data/202058/000020205824000168/hrs-20240927.htm',
 'https://www.sec.gov/Archives/edgar/data/202058/000020205824000140/hrs-20240628.htm']

In [ ]:
corpus_ = get_report_corpus('https://www.sec.gov/Archives/edgar/data/202058/000020205824000140/hrs-20240627.htm')

In [ ]:
get_ids(corpus_)

IndexError: list index out of range

In [ ]:
mdaf = corpus.find_all('span')

for tag in mdaf:
    if tag.text.upper()=='ITEM 2.': #
        print(tag)
        break


<span style="color:#0000ff;font-family:'Helvetica',sans-serif;font-size:9pt;font-weight:400;line-height:100%;text-decoration:underline"><a href="#i056866be11a54295b8c21e0877b67331_67" style="color:#0000ff;font-family:'Helvetica',sans-serif;font-size:9pt;font-weight:400;line-height:100%;text-decoration:underline">Item 2.</a></span>
